In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install fastparquet
!pip install yfinance
!pip install seaborn

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from fastparquet import ParquetFile
from plotly.offline import plot, iplot  
from tqdm.notebook import tqdm, trange
from sklearn.preprocessing import StandardScaler

import math
import time
import plotly.graph_objs as go
import plotly.offline as py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
import seaborn as sns

#Panda Config
pd.set_option('display.max_rows', 1000)

In [3]:
dtf_full = ParquetFile('/content/drive/MyDrive/Colab Notebooks/I2A2/database.parquet').to_pandas()

In [4]:
dtf_full['Ticker_Exchange'] = dtf_full['Ticker'] + '_' + dtf_full['Exchange']
dtf_full = dtf_full[(dtf_full['Open'] != 0)]
dtf_full = dtf_full[(dtf_full['Close'] != 0)]
dtf_full['Open_log'] = np.log10(dtf_full['Open'])
dtf_full['Close_log'] = np.log10(dtf_full['Close'])
dtf_full['Diff'] = dtf_full['Close_log'] - dtf_full['Open_log']
dtf_full['Diff_norm'] = (dtf_full['Diff'] - dtf_full['Diff'].min()) / (dtf_full['Diff'].max() - dtf_full['Diff'].min())
dtf_full.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,Exchange,Ticker_Exchange,Open_log,Close_log,Diff,Diff_norm
0,2008-09-02,25.100143,25.536482,24.527897,24.670959,22.517595,2876665.0,A,NYSE,A_NYSE,1.399676,1.392186,-0.007490,0.476271
1,2008-09-03,24.570814,24.678112,23.862661,24.184549,22.073639,3855265.0,A,NYSE,A_NYSE,1.390420,1.383538,-0.006882,0.476368
2,2008-09-04,23.977110,23.984262,22.989986,23.018599,21.009457,3656609.0,A,NYSE,A_NYSE,1.379797,1.362079,-0.017718,0.474643
3,2008-09-05,22.889843,23.547926,22.711016,23.454935,21.407715,3217357.0,A,NYSE,A_NYSE,1.359643,1.370234,0.010591,0.479148
4,2008-09-08,23.876966,23.977110,23.297567,23.583691,21.525227,3046102.0,A,NYSE,A_NYSE,1.377979,1.372612,-0.005367,0.476609


In [10]:
dtf_patter = dtf_full[(dtf_full['Date'] >= '2021-12-01') & (dtf_full['Date'] < '2022-02-02') & (dtf_full['Ticker_Exchange']=='TD_TO')]

trace = go.Candlestick(x=dtf_patter['Date'],
                       open=dtf_patter['Open'],
                       high=dtf_patter['High'],
                       low=dtf_patter['Low'],
                       close=dtf_patter['Close'])
data = [trace]
py.iplot(data, filename="Pattern Candle")

KeyError: ignored

In [6]:
#Remove unnecessary columns
remove_columns = ["Adj Close",  "Volume", "Ticker", "Exchange", "High", "Low", "Open", "Close"]

for column in remove_columns:
  dtf_full.drop(column, inplace=True, axis=1)

dtf_full.head()

,Date,Ticker_Exchange,Open_log,Close_log,Diff,Diff_norm
0,2008-09-02,A_NYSE,1.399676,1.392186,-0.007490,0.476271
1,2008-09-03,A_NYSE,1.390420,1.383538,-0.006882,0.476368
2,2008-09-04,A_NYSE,1.379797,1.362079,-0.017718,0.474643
3,2008-09-05,A_NYSE,1.359643,1.370234,0.010591,0.479148
4,2008-09-08,A_NYSE,1.377979,1.372612,-0.005367,0.476609


In [22]:
tickers = dtf_full['Ticker_Exchange'].unique()
tickers

6

In [28]:
df = pd.DataFrame()
df = df.append(dtf_patter)
print(df.shape)
df = df.append(dtf_full[(dtf_full['Date'] >= '2021-12-01') & (dtf_full['Date'] < '2022-02-02')])
print(df.shape)
remove_columns = ["Open_log", "Close_log", "Diff"]

for column in remove_columns:
  df.drop(column, inplace=True, axis=1)

df['Ticker_Exchange'].unique()

(42, 6)
(376443, 6)


array(['TD_TO', 'A_NYSE', 'AA_NYSE', ..., 'ZYXI_NASDAQ', 'ZZZ_TO',
       'ZZZD_TO'], dtype=object)

In [34]:
df_pearson = pd.DataFrame()
column = 'Diff_norm'

for ticker_ in tqdm(tickers):
    df_pearson[ticker_] = (df.loc[df['Ticker_Exchange'] == ticker_, column]).reset_index(drop=True)
    
df_pearson.head(100)

  0%|          | 0/8932 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`



,A_NYSE,AA_NYSE,AAB_TO,AAC_NYSE,AACG_NASDAQ,AACI_NASDAQ,AADI_NASDAQ,AAIC_NYSE,AAIN_NYSE,AAL_NASDAQ,...,ZWS_TO,ZWT_TO,ZWU_TO,ZXM_TO,ZY_NASDAQ,ZYME_NYSE,ZYNE_NASDAQ,ZYXI_NASDAQ,ZZZ_TO,ZZZD_TO
0,0.476119,0.471261,0.477463,0.477463,0.468812,0.476760,0.473485,0.476128,0.477460,0.470752,...,0.475980,0.475658,0.476681,0.476879,0.471719,0.477702,0.473803,0.476341,0.475803,0.477637
1,0.478887,0.478059,0.477463,0.477322,0.472571,0.477427,0.477947,0.479179,0.477507,0.481255,...,0.478369,0.478346,0.478021,0.477479,0.478348,0.477258,0.478086,0.475827,0.478212,0.477513
2,0.476320,0.476389,0.477463,0.477533,0.465475,0.477533,0.474282,0.477274,0.477296,0.474811,...,0.477815,0.476732,0.477629,0.477463,0.468143,0.471008,0.474287,0.473441,0.476641,0.477984
3,0.478442,0.478441,0.477463,0.477463,0.483703,0.477463,0.481451,0.477842,0.477323,0.481348,...,0.478036,0.478412,0.477904,0.477463,0.482405,0.477387,0.478963,0.480280,0.478394,0.477687
4,0.477896,0.477272,0.472341,0.477463,0.484595,0.477392,0.478188,0.476517,0.477295,0.475821,...,0.477526,0.477850,0.477518,0.478072,0.480114,0.476859,0.479104,0.479002,0.478483,0.478159
5,0.478152,0.479617,0.477463,0.477604,0.473675,0.477463,0.479540,0.478416,0.477463,0.478494,...,0.477621,0.477964,0.477135,0.477510,0.478638,0.479611,0.480252,0.477136,0.476973,0.477635
6,0.476623,0.480545,0.477463,0.477181,0.483437,0.477392,0.479881,0.478403,0.477323,0.477845,...,0.477463,0.476847,0.477133,0.477463,0.473826,0.467017,0.476271,0.475401,0.477591,0.477315
7,0.478232,0.475825,0.477463,0.477463,0.474028,0.477534,0.478324,0.476903,0.477253,0.477233,...,0.477840,0.477329,0.477573,0.477479,0.475210,0.475303,0.474408,0.471116,0.476290,0.477463
8,0.477000,0.475410,0.482585,0.477533,0.470522,0.477463,0.481177,0.476895,0.477463,0.475748,...,0.477431,0.476735,0.477628,0.477318,0.480452,0.476464,0.477463,0.470482,0.475261,0.477562
9,0.476106,0.479549,0.477463,0.477392,0.477463,0.477463,0.479566,0.476311,0.477463,0.477832,...,0.477936,0.477230,0.477628,0.477463,0.475538,0.478779,0.476183,0.476105,0.477192,0.478007


In [40]:
first_column = df_pearson.pop('TD_TO')
df_pearson.insert(0, 'TD_TO', first_column)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2828: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`



In [41]:
df_pearson = df_pearson.fillna(0)
df_pearson.tail(1)

,TD_TO,A_NYSE,AA_NYSE,AAB_TO,AAC_NYSE,AACG_NASDAQ,AACI_NASDAQ,AADI_NASDAQ,AAIC_NYSE,AAIN_NYSE,...,ZWS_TO,ZWT_TO,ZWU_TO,ZXM_TO,ZY_NASDAQ,ZYME_NYSE,ZYNE_NASDAQ,ZYXI_NASDAQ,ZZZ_TO,ZZZD_TO
42,0.477298,0.477708,0.477677,0.0,0.477676,0.480985,0.478023,0.477137,0.478271,0.477435,...,0.0,0.0,0.0,0.0,0.476522,0.47293,0.479034,0.476853,0.0,0.0


In [42]:
df_corr = df_pearson.corr(method='pearson')
df_corr.head()

,TD_TO,A_NYSE,AA_NYSE,AAB_TO,AAC_NYSE,AACG_NASDAQ,AACI_NASDAQ,AADI_NASDAQ,AAIC_NYSE,AAIN_NYSE,...,ZWS_TO,ZWT_TO,ZWU_TO,ZXM_TO,ZY_NASDAQ,ZYME_NYSE,ZYNE_NASDAQ,ZYXI_NASDAQ,ZZZ_TO,ZZZD_TO
TD_TO,1.000000,0.361783,0.107890,0.089702,-0.140304,-0.105258,-0.044755,0.060083,0.080907,0.047356,...,0.082351,0.084079,0.081170,0.080588,0.268177,0.271404,0.320611,0.207284,0.082682,0.081290
A_NYSE,0.361783,1.000000,0.367170,-0.054198,-0.001443,-0.276617,0.070036,0.377788,0.368994,-0.079770,...,-0.057064,-0.054291,-0.057697,-0.057237,0.571380,0.267306,0.552010,0.409385,-0.056878,-0.058970
AA_NYSE,0.107890,0.367170,1.000000,-0.003635,-0.112622,-0.012331,0.115559,0.384724,0.357920,-0.013336,...,-0.001636,-0.001937,-0.002318,-0.002872,0.366231,0.014303,0.490789,0.307769,-0.000605,-0.002834
AAB_TO,0.089702,-0.054198,-0.003635,1.000000,-0.291344,-0.055613,-0.425500,0.027035,-0.100012,0.025138,...,0.998791,0.998749,0.998772,0.998670,0.010868,0.158416,-0.108715,0.005118,0.998695,0.998765
AAC_NYSE,-0.140304,-0.001443,-0.112622,-0.291344,1.000000,0.222739,0.195534,-0.129335,0.057718,0.007541,...,-0.280751,-0.280498,-0.279954,-0.278900,0.049883,0.206270,0.000642,0.042941,-0.281470,-0.279950


In [43]:
df_corr_sorted = df_corr.sort_values(by='TD_TO', ascending=False)

df_open_best = df_corr_sorted.loc[df_corr_sorted['TD_TO'] < 1, 'TD_TO']

df_open_best_10 = df_open_best.head(10)

print(df_open_best_10)

NXST_NASDAQ    0.662700
SSP_NASDAQ     0.604976
UI_NYSE        0.595190
AKUS_NASDAQ    0.585211
WY_NYSE        0.581276
KAR_NYSE       0.572313
SMMF_NASDAQ    0.569725
EVC_NYSE       0.567598
ACHC_NASDAQ    0.558585
MAN_NYSE       0.554873
Name: TD_TO, dtype: float64


In [44]:
stocks = list(df_open_best_10.index)

stocks.append('TD_TO')

plot.rcParams.update({'figure.figsize': (10,10)})

df_pearson.plot.line(y=stocks, title="Open")

plot.show(block=True)

AttributeError: ignored

In [27]:
#Verificando se existem valores nulos
#Identificado que não existem valores nulos

# dtf_full.isnull()
# dtf_full.dtypes
# dtf_full.info()
# dtf_full.isnull().sum()
#pegar valores entre 01/12/21 até 02/02/2022 --> 63
# dtf_full.shape[]
# dtf_full.set_index('Date')




Bibliografia: 

https://www.delftstack.com/pt/howto/numpy/calculate-euclidean-distance/

tirar colunas inuteis (feito)

para cada col, col - col.min() / col.max-col.min

coorelação de pearson(usar essa)

Média movel

distancia euclidiana

PCA
